#Телеграм бот с нейронкой
Здесь я попытался навайбкодить тг бота. Можете тоже поробовать его запустить, для этого выполните все ячейки. Первая ячейка устнавливает библиотеки.Вторая - тело, код бота. Третья - запуск сеанса. Можете нажать 'Выполнить все', и подождать когда запуститься 3 ячейка. Когда это случиться, можете писать боту @lumen52_bot. Когда захотите прервать сеанс, прервите 3 ячейку. Отмечу, что вся настройка ботов(в принципе), реализутеся через код и через бота BotFather(ес че, официальный бот тг, позволяющий создавать других ботов).

In [1]:
!pip install python-telegram-bot --upgrade
!pip install gigachat
!pip install -U langchain-gigachat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.0/731.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.2 MB/s eta 0:00:00


In [2]:
import asyncio
import logging
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain_gigachat.chat_models import GigaChat

# === Настройка GigaChat ===
credentials = "MDE5OTc4MTEtM2NjMS03ODNkLTkxYzAtMmM4MzZhN2UxNzM2OmE0YmY3NTdkLTkxNjItNGRjNi04ZDA1LTBiOTM4ZTRjM2JjOA=="
giga = GigaChat(
    model="GigaChat-2-Max",
    credentials=credentials,
    verify_ssl_certs=False
)

# Промпт (без return_messages!)
prompt_template = """Ты бот, который создает планы обучения для людей. Учитывай контекст предыдущих сообщений. При запросе пользователя плана обучения, учитывай требования к плану обучения от пользователя. План должен выглядеть в виде таблицы.
{history}
Human: {input}
Assistant:"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=prompt_template)

# Хранилище цепочек (по user_id)
user_chains = {}

def get_chain(user_id: int) -> ConversationChain:
    if user_id not in user_chains:
        memory = ConversationBufferMemory()  # ← без return_messages!
        chain = ConversationChain(
            llm=giga,
            memory=memory,
            prompt=PROMPT,
            verbose=False
        )
        user_chains[user_id] = chain
    return user_chains[user_id]

# === Telegram обработчики ===
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Привет! Я помогу составить план обучения 📚\n"
        "Напишите, что хотите изучать и за какое время."
    )

async def clear(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.effective_user.id
    if user_id in user_chains:
        del user_chains[user_id]
    await update.message.reply_text("Память очищена!")

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.effective_user
    text = update.message.text

    try:
        await update.message.reply_text("Готовлю план... ⏳")
        chain = get_chain(user.id)

        # Выполняем синхронный вызов в отдельном потоке
        loop = asyncio.get_event_loop()
        response = await loop.run_in_executor(None, chain.predict, text)

        await update.message.reply_text(response.strip())
    except Exception as e:
        logging.exception("Ошибка в обработке сообщения")
        await update.message.reply_text("Извините, произошла ошибка. Попробуйте позже.")

# === Запуск ===
async def main():
    TOKEN = "8250825015:AAE9nIh5RLmbjNFl2yS0m3sBUKhfi3VJXd8"  # ← замените!
    app = Application.builder().token(TOKEN).build()

    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("clear", clear))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    logging.info("Бот запущен...")
    await app.initialize()
    await app.start()
    await app.updater.start_polling()

    try:
        while True:
            await asyncio.sleep(10)
    except KeyboardInterrupt:
        pass
    finally:
        await app.stop()
        await app.shutdown()

In [3]:
await main()

/tmp/ipython-input-202424893.py:31: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()  # ← без return_messages!
/tmp/ipython-input-202424893.py:32: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(
ERROR:root:Ошибка в обработке сообщения
Traceback (most recent call last):
  File "/tmp/ipython-input-202424893.py", line 64, in handle_message
    response = await loop.run_in_executor(None, chain.predict, text)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/concurrent/futures/thread.py", line 59, in run
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langc

RuntimeError: This Updater is still running!